| Description

Authors of notebook: Mateus Balda and Alessandro Bof

Reference paper and dataset: https://www.frontiersin.org/journals/psychiatry/articles/10.3389/fpsyt.2021.707581/full

| Notebook structure
0. Setup and Imports
1. Utility Functions  
2. Data Loading and Preprocessing
3. Data Preparation for Training
4. Model Definition
5. Training and Evaluation
6. Results
7. Conclusions, Problems, and Limitations

| Training
1. Multiclass classification for `y` main.disorder
2. Synthetic data (CTGAN)
3. Training all features

## | 0. Setup and Imports

In [1]:
!pip install -q pytorch_tabnet;

In [2]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np

from IPython.display import display, HTML
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve, roc_auc_score
from sklearn.metrics import confusion_matrix
from sklearn.impute import KNNImputer

import torch
import torch.nn as nn
import torch.nn.init as init
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.optim.lr_scheduler import ReduceLROnPlateau
from sklearn.decomposition import PCA
from sklearn.model_selection import StratifiedKFold

from warnings import filterwarnings
filterwarnings('ignore')

torch.__version__

'1.13.1+cu117'

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [4]:
np.random.seed(123)
torch.manual_seed(123)

## | 1. Utility Functions

### 1. OUTLIERS

In [5]:
def detect_outliers_summary(df):
    outliers_summary = {}

    for col in df.select_dtypes(include=['float64', 'int64']).columns:
        Q1 = df[col].quantile(0.25)
        Q3 = df[col].quantile(0.75)
        IQR = Q3 - Q1
        lower_limit = Q1 - 1.5 * IQR
        upper_limit = Q3 + 1.5 * IQR

        outliers = df[(df[col] < lower_limit) | (df[col] > upper_limit)][col]
        
        outliers_summary[col] = {
            'num_outliers': len(outliers),
            'percent_outliers': len(outliers) / len(df) * 100,
            'outliers': outliers.tolist(),
            'lower_limit': lower_limit,
            'upper_limit': upper_limit
        }

    return pd.DataFrame(outliers_summary).T


def treat_all_outliers_iqr(df, factor=1.5):
    df_treated = df.copy()
    
    for column in df_treated.select_dtypes(include=[np.number]).columns:
        Q1 = np.percentile(df_treated[column], 25)
        Q3 = np.percentile(df_treated[column], 75)
        IQR = Q3 - Q1

        lower_bound = Q1 - factor * IQR
        upper_bound = Q3 + factor * IQR

        df_treated[column] = np.clip(df_treated[column], lower_bound, upper_bound)

    return df_treated

### 2. NANS

In [6]:
def remove_missing_columns(df, threshold=0.5):
    limit = int(threshold * len(df))
    df = df.dropna(thresh=limit, axis=1)
    return df
    
def find_most_null_column(df, threshold=0.5):
    null_ratios = df.isnull().mean()
    for col, ration in null_ratios.items():
        if ration > threshold:
            return col
    return None

def analyze_missing_values(df):
    missing_values = df.isnull().sum()
    missing_values = missing_values[missing_values > 0]
    total_number_nans = df.isnull().sum().sum()
    
    return missing_values, total_number_nans

def handle_nans(df):
    columns_with_nans = df.columns[df.isnull().any()].tolist()
    
    knn_imputer = KNNImputer(n_neighbors=5, weights='uniform', metric='nan_euclidean')
    
    df_imputed = pd.DataFrame(knn_imputer.fit_transform(df[columns_with_nans]),
                                columns=columns_with_nans)
    
    df[columns_with_nans] = df_imputed[columns_with_nans]
    
    return df

## | 2. Data Loading and Preprocessing

In [7]:
df = pd.read_csv('../input/eeg-psychiatric-disorders-dataset/EEG.machinelearing_data_BRMH.csv')
df_synthetic = pd.read_csv('../input/synthetic-samples-main-disorder-ctgan/synthetic_samples_main_disorder_ctgan.csv', index_col=0)

df.shape, df_synthetic.shape

((945, 1149), (917, 1144))

In [8]:
df_synthetic.head()

,age,education,IQ,AB.A.delta.a.FP1,AB.A.delta.b.FP2,AB.A.delta.c.F7,AB.A.delta.d.F3,AB.A.delta.e.Fz,AB.A.delta.f.F4,AB.A.delta.g.F8,...,COH.F.gamma.o.Pz.q.T6,COH.F.gamma.o.Pz.r.O1,COH.F.gamma.o.Pz.s.O2,COH.F.gamma.p.P4.q.T6,COH.F.gamma.p.P4.r.O1,COH.F.gamma.p.P4.s.O2,COH.F.gamma.q.T6.r.O1,COH.F.gamma.q.T6.s.O2,COH.F.gamma.r.O1.s.O2,main.disorder
0,23.376377,12.043075,112.643365,27.144192,16.187808,12.907181,23.669069,29.272920,30.599443,19.087608,...,85.181915,36.457220,78.339581,71.082923,71.358454,1.510461,21.019587,69.051350,97.160001,Addictive disorder
1,20.786404,17.043296,113.232441,10.172036,13.424233,21.840080,23.491504,32.526876,34.392703,13.730551,...,73.484918,47.733036,88.327693,76.842599,103.759471,72.031505,4.999746,85.836417,116.402084,Addictive disorder
2,62.353295,13.435671,65.597080,14.793148,17.651986,15.991087,25.001954,14.233273,12.183853,22.903653,...,45.342838,30.433142,80.407288,93.064088,92.449444,27.130648,-6.309497,97.717736,93.094953,Addictive disorder
3,19.308306,12.404831,117.544206,9.177333,6.731468,4.443113,21.096932,7.735151,36.469104,25.138482,...,88.210760,56.262836,40.570765,51.369780,80.274405,23.631445,7.374698,67.858076,95.123691,Addictive disorder
4,23.073199,15.074009,60.646940,27.206730,3.856561,18.725702,22.135223,16.158441,8.871639,18.606874,...,57.424337,52.391691,84.822487,79.001387,57.053102,32.296730,22.092730,103.311152,64.631042,Addictive disorder


In [9]:
# Checking / imputing Nan

output1 = analyze_missing_values(df)
output2 = find_most_null_column(df)
df = remove_missing_columns(df)
output3 = analyze_missing_values(df)
df = handle_nans(df)

display(output1)
display(HTML('<hr>'))
display(output2)
display(HTML('<hr>'))
display(output3)
display(df.shape)
display(HTML('<hr>'))
display(df.isna().sum().sum())
display(df)

(education        15
 IQ               13
 Unnamed: 122    945
 dtype: int64,
 973)

'Unnamed: 122'

(education    15
 IQ           13
 dtype: int64,
 28)

(945, 1148)

0

,no.,sex,age,eeg.date,education,IQ,main.disorder,specific.disorder,AB.A.delta.a.FP1,AB.A.delta.b.FP2,...,COH.F.gamma.o.Pz.p.P4,COH.F.gamma.o.Pz.q.T6,COH.F.gamma.o.Pz.r.O1,COH.F.gamma.o.Pz.s.O2,COH.F.gamma.p.P4.q.T6,COH.F.gamma.p.P4.r.O1,COH.F.gamma.p.P4.s.O2,COH.F.gamma.q.T6.r.O1,COH.F.gamma.q.T6.s.O2,COH.F.gamma.r.O1.s.O2
0,1,M,57.0,2012.8.30,13.43871,101.580472,Addictive disorder,Alcohol use disorder,35.998557,21.717375,...,55.989192,16.739679,23.452271,45.678820,30.167520,16.918761,48.850427,9.422630,34.507082,28.613029
1,2,M,37.0,2012.9.6,6.00000,120.000000,Addictive disorder,Alcohol use disorder,13.425118,11.002916,...,45.595619,17.510824,26.777368,28.201062,57.108861,32.375401,60.351749,13.900981,57.831848,43.463261
2,3,M,32.0,2012.9.10,16.00000,113.000000,Addictive disorder,Alcohol use disorder,29.941780,27.544684,...,99.475453,70.654171,39.131547,69.920996,71.063644,38.534505,69.908764,27.180532,64.803155,31.485799
3,4,M,35.0,2012.10.8,18.00000,126.000000,Addictive disorder,Alcohol use disorder,21.496226,21.846832,...,59.986561,63.822201,36.478254,47.117006,84.658376,24.724096,50.299349,35.319695,79.822944,41.141873
4,5,M,36.0,2012.10.18,16.00000,112.000000,Addictive disorder,Alcohol use disorder,37.775667,33.607679,...,61.462720,59.166097,51.465531,58.635415,80.685608,62.138436,75.888749,61.003944,87.455509,70.531662
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
940,941,M,22.0,2014.8.28,13.00000,116.000000,Healthy control,Healthy control,41.851823,36.771496,...,82.905657,34.850706,63.970519,63.982003,51.244725,62.203684,62.062237,31.013031,31.183413,98.325230
941,942,M,26.0,2014.9.19,13.00000,118.000000,Healthy control,Healthy control,18.986856,19.401387,...,65.917918,66.700117,44.756285,49.787513,98.905995,54.021304,93.902401,52.740396,92.807331,56.320868
942,943,M,26.0,2014.9.27,16.00000,113.000000,Healthy control,Healthy control,28.781317,32.369230,...,61.040959,27.632209,45.552852,33.638817,46.690983,19.382928,41.050717,7.045821,41.962451,19.092111
943,944,M,24.0,2014.9.20,13.00000,107.000000,Healthy control,Healthy control,19.929100,25.196375,...,99.113664,48.328934,41.248470,28.192238,48.665743,42.007147,28.735945,27.176500,27.529522,20.028446


In [10]:
X = df.iloc[:,8:]
target_name = 'main.disorder'
target = df[target_name]
quantitative_features = df.loc[:, ['age', 'education', 'IQ']]

assert X.shape[0] == target.shape[0] == quantitative_features.shape[0], "Inconsistent number of samples across datasets"

X.shape, target.shape, quantitative_features.shape

((945, 1140), (945,), (945, 3))

In [11]:
df_real = pd.concat([quantitative_features, X, target], axis=1)

df_combined = pd.concat([df_real, df_synthetic], axis=0).reset_index(drop=True)

X = df_combined.iloc[:, 3:-1]
quantitative_features = df_combined.loc[:, ['age', 'education', 'IQ']]
target = df_combined[target_name]
X_concatenated = pd.concat([quantitative_features, X], axis=1)

assert target.index.equals(X.index) and target.index.equals(X_concatenated.index), "Indices of target and features do not match"

X.shape, target.shape, quantitative_features.shape, X_concatenated.shape

((1862, 1140), (1862,), (1862, 3), (1862, 1143))

In [12]:
CLASS_NAMES = np.unique(target).tolist()
CLASS_NAMES

['Addictive disorder',
 'Anxiety disorder',
 'Healthy control',
 'Mood disorder',
 'Obsessive compulsive disorder',
 'Schizophrenia',
 'Trauma and stress related disorder']

In [13]:
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(target)
y_encoded = pd.Series(y_encoded, name=target_name)
(pd.DataFrame(y_encoded)).value_counts()

main.disorder
0                266
1                266
2                266
3                266
4                266
5                266
6                266
Name: count, dtype: int64

In [14]:
df_union = pd.concat([X_concatenated, pd.DataFrame(y_encoded)], axis=1)

assert df_union.shape[0] == X_concatenated.shape[0] == y_encoded.shape[0], "Inconsistent number of samples in the union DataFrame"
assert X_concatenated.index.equals(y_encoded.index), "Indices of features and target do not match in the union DataFrame"

display(df_union.shape)
display(df_union)
display(df_union.isna().sum().sum())

(1862, 1144)

,age,education,IQ,AB.A.delta.a.FP1,AB.A.delta.b.FP2,AB.A.delta.c.F7,AB.A.delta.d.F3,AB.A.delta.e.Fz,AB.A.delta.f.F4,AB.A.delta.g.F8,...,COH.F.gamma.o.Pz.q.T6,COH.F.gamma.o.Pz.r.O1,COH.F.gamma.o.Pz.s.O2,COH.F.gamma.p.P4.q.T6,COH.F.gamma.p.P4.r.O1,COH.F.gamma.p.P4.s.O2,COH.F.gamma.q.T6.r.O1,COH.F.gamma.q.T6.s.O2,COH.F.gamma.r.O1.s.O2,main.disorder
0,57.000000,13.438710,101.580472,35.998557,21.717375,21.518280,26.825048,26.611516,25.732649,16.563408,...,16.739679,23.452271,45.678820,30.167520,16.918761,48.850427,9.422630,34.507082,28.613029,0
1,37.000000,6.000000,120.000000,13.425118,11.002916,11.942516,15.272216,14.151570,12.456034,8.436832,...,17.510824,26.777368,28.201062,57.108861,32.375401,60.351749,13.900981,57.831848,43.463261,0
2,32.000000,16.000000,113.000000,29.941780,27.544684,17.150159,23.608960,27.087811,13.541237,16.523963,...,70.654171,39.131547,69.920996,71.063644,38.534505,69.908764,27.180532,64.803155,31.485799,0
3,35.000000,18.000000,126.000000,21.496226,21.846832,17.364316,13.833701,14.100954,13.100939,14.613650,...,63.822201,36.478254,47.117006,84.658376,24.724096,50.299349,35.319695,79.822944,41.141873,0
4,36.000000,16.000000,112.000000,37.775667,33.607679,21.865556,21.771413,22.854536,21.456377,15.969042,...,59.166097,51.465531,58.635415,80.685608,62.138436,75.888749,61.003944,87.455509,70.531662,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1857,56.586838,15.910817,92.598793,58.344625,16.250204,22.165892,3.017798,1.382935,23.911295,6.217182,...,9.107626,20.435264,63.706446,66.611018,25.983391,57.757929,56.713252,67.893062,96.850570,4
1858,34.766892,16.829058,69.452233,26.380735,38.919817,19.233551,22.543899,17.145224,2.036416,-2.938821,...,40.756644,84.770737,93.419613,71.142697,89.953221,70.565893,48.294927,39.524151,86.316144,4
1859,25.948962,13.414100,64.088265,6.856573,21.632333,21.378422,9.903213,51.725818,26.871169,6.390204,...,84.125462,85.274250,51.210326,10.838181,13.540271,69.141473,63.682173,68.032213,61.115077,4
1860,35.215274,18.050538,67.202613,4.865726,21.731444,21.325605,31.383891,39.464086,16.740952,13.293345,...,45.337457,14.478245,81.563188,76.144551,61.625960,52.572968,93.442888,54.596480,27.025361,4


0

In [15]:
# detecting outliers

outliers_summary = detect_outliers_summary(df_union)
outliers_summary.to_csv('outliers_summary.csv', index=True)
outliers_summary['num_outliers'].sum()

56613

## | 3. Data Preparation for Training

In [16]:
# X_concatenated
# y_encoded

In [17]:
TRAIN_RATIO = 0.7
VAL_RATIO = 0.15

CRITERION = nn.CrossEntropyLoss()
SCALER = StandardScaler()

NUM_EPOCHS = 500

## | 4. Model Definition

In [18]:
from pytorch_tabnet.tab_model import TabNetClassifier

In [19]:
clf = TabNetClassifier(
    optimizer_fn=torch.optim.Adam,
    scheduler_params={"step_size":30, 
                        "gamma":0.9},
    scheduler_fn=torch.optim.lr_scheduler.StepLR,
    verbose=5,
)

## | 5. Training and Evaluation  

In [20]:
X_concatenated.shape[1]
len(np.unique(y_encoded))

7

In [21]:
def print_accuracy_report(accuracy, y_true, y_pred, class_names, verbose=True):
    report = classification_report(y_true, y_pred, target_names=class_names, digits=4, output_dict=True)
    cm = confusion_matrix(y_true, y_pred)
    
    sensitivities = {}
    specificities = {}

    for i, class_name in enumerate(class_names):
        TP = cm[i, i]
        FN = np.sum(cm[i, :]) - TP
        FP = np.sum(cm[:, i]) - TP
        TN = np.sum(cm) - (TP + FP + FN)

        sensitivity = TP / (TP + FN) if (TP + FN) != 0 else 0.0
        specificity = TN / (TN + FP) if (TN + FP) != 0 else 0.0

        sensitivities[class_name] = sensitivity
        specificities[class_name] = specificity

    if verbose:
        print(f"\n-- Test Accuracy: {accuracy:.2f}%")
        print("\n-- Classification Report:\n", classification_report(y_true, y_pred, target_names=class_names, digits=4))

        print("-- Sensitivity (Recall) por classe:")
        for cls, sens in sensitivities.items():
            print(f"  - {cls}: {sens:.4f}")

        print("\n-- Specificity por classe:")
        for cls, spec in specificities.items():
            print(f"  - {cls}: {spec:.4f}")

    return report, sensitivities, specificities

In [22]:
from sklearn.metrics import accuracy_score

num_folds = 5
skf = StratifiedKFold(n_splits=num_folds, shuffle=True, random_state=42)

def cross_validate_model(
    X, y, 
    class_names, num_classes
):
    accuracies = []
    aucs = []
    sensitivities = []
    specificities = []

    for fold, (train_index, test_index) in enumerate(skf.split(X, y)):
        print(f"\n- Fold {fold + 1}/{num_folds}")
        
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]
        
        X_train_scaled = SCALER.fit_transform(X_train)
        X_test_scaled = SCALER.transform(X_test)
        
        y_train_array = y_train.to_numpy()
        y_test_array = y_test.to_numpy()
        
        clf.fit(
            X_train_scaled, y_train_array,
            eval_set=[(X_train_scaled, y_train_array), (X_test_scaled, y_test_array)],
            eval_name=['train', 'test'],
            eval_metric=['accuracy'],
            max_epochs=200, patience=60,
            batch_size=48, virtual_batch_size=24,
            num_workers=0,
            weights=1,
            drop_last=True
        )
        
        y_pred = clf.predict(X_test_scaled)
        y_proba = clf.predict_proba(X_test_scaled)

        acc = accuracy_score(y_test_array, y_pred)

        report, fold_sensitivities, fold_specificities = print_accuracy_report(
            acc, y_test_array, y_pred, class_names
        )

        auc = roc_auc_score(y_test_array, y_proba, multi_class='ovr', average='macro', labels=np.arange(num_classes))
        
        accuracies.append(acc)
        aucs.append(auc)
        sensitivities.append(fold_sensitivities)
        specificities.append(fold_specificities)

    # Calculate mean and std for all metrics across folds
    mean_accuracy = np.mean(accuracies)
    std_accuracy = np.std(accuracies)
    mean_auc = np.mean(aucs)
    std_auc = np.std(aucs)
    
    mean_sens = {cls: np.mean([s[cls] for s in sensitivities]) for cls in class_names}
    std_sens = {cls: np.std([s[cls] for s in sensitivities]) for cls in class_names}
    
    mean_spec = {cls: np.mean([s[cls] for s in specificities]) for cls in class_names}
    std_spec = {cls: np.std([s[cls] for s in specificities]) for cls in class_names}
    
    return {
        'mean_accuracy': mean_accuracy,
        'std_accuracy': std_accuracy,
        'mean_auc': mean_auc,
        'std_auc': std_auc,
        'mean_sens': mean_sens,
        'std_sens': std_sens,
        'mean_spec': mean_spec,
        'std_spec': std_spec
    }

In [23]:
cv_results = cross_validate_model(
    X=X_concatenated, 
    y=y_encoded,
    class_names=CLASS_NAMES,
    num_classes=len(np.unique(y_encoded)),
)


- Fold 1/5
epoch 0  | loss: 2.13269 | train_accuracy: 0.1679  | test_accuracy: 0.14477 |  0:00:01s
epoch 5  | loss: 1.22997 | train_accuracy: 0.57152 | test_accuracy: 0.59517 |  0:00:07s
epoch 10 | loss: 1.07837 | train_accuracy: 0.6266  | test_accuracy: 0.64879 |  0:00:13s
epoch 15 | loss: 1.03914 | train_accuracy: 0.61988 | test_accuracy: 0.64075 |  0:00:18s
epoch 20 | loss: 0.97793 | train_accuracy: 0.62794 | test_accuracy: 0.65684 |  0:00:24s
epoch 25 | loss: 0.99164 | train_accuracy: 0.63197 | test_accuracy: 0.65416 |  0:00:29s
epoch 30 | loss: 0.94008 | train_accuracy: 0.63465 | test_accuracy: 0.65952 |  0:00:35s
epoch 35 | loss: 0.9261  | train_accuracy: 0.64943 | test_accuracy: 0.64879 |  0:00:41s
epoch 40 | loss: 0.92697 | train_accuracy: 0.65346 | test_accuracy: 0.64075 |  0:00:46s
epoch 45 | loss: 0.88996 | train_accuracy: 0.66286 | test_accuracy: 0.66488 |  0:00:52s
epoch 50 | loss: 0.92035 | train_accuracy: 0.67226 | test_accuracy: 0.67292 |  0:00:57s
epoch 55 | loss: 0.8

## | 6. Results

In [24]:
# dataframe 
cv_df = pd.DataFrame({
    'Metric': ['Mean Accuracy', 'Std Accuracy', 'Mean AUC', 'Std AUC'] + 
              [f'Mean Sensitivity ({cls})' for cls in CLASS_NAMES] + 
              [f'Std Sensitivity ({cls})' for cls in CLASS_NAMES] + 
              [f'Mean Specificity ({cls})' for cls in CLASS_NAMES] + 
              [f'Std Specificity ({cls})' for cls in CLASS_NAMES],
    'Value': [
        cv_results['mean_accuracy'], cv_results['std_accuracy'],
        cv_results['mean_auc'], cv_results['std_auc']
    ] + 
    [cv_results['mean_sens'][cls] for cls in CLASS_NAMES] + 
    [cv_results['std_sens'][cls] for cls in CLASS_NAMES] + 
    [cv_results['mean_spec'][cls] for cls in CLASS_NAMES] + 
    [cv_results['std_spec'][cls] for cls in CLASS_NAMES]
})

cv_df

,Metric,Value
0,Mean Accuracy,0.664333
1,Std Accuracy,0.016406
2,Mean AUC,0.893943
3,Std AUC,0.009720
4,Mean Sensitivity (Addictive disorder),0.548847
5,Mean Sensitivity (Anxiety disorder),0.597624
6,Mean Sensitivity (Healthy control),0.770510
7,Mean Sensitivity (Mood disorder),0.796646
8,Mean Sensitivity (Obsessive compulsive disorder),0.826904
9,Mean Sensitivity (Schizophrenia),0.559888


In [25]:
rows = []

rows.append({
    'Class': 'Overall',
    'Metric': 'Accuracy',
    'Mean': cv_results['mean_accuracy'],
    'Std': cv_results['std_accuracy']
})
rows.append({
    'Class': 'Overall',
    'Metric': 'AUC',
    'Mean': cv_results['mean_auc'],
    'Std': cv_results['std_auc']
})

for cls in CLASS_NAMES:
    rows.append({
        'Class': cls,
        'Metric': 'Sensitivity',
        'Mean': cv_results['mean_sens'][cls],
        'Std': cv_results['std_sens'][cls]
    })
    rows.append({
        'Class': cls,
        'Metric': 'Specificity',
        'Mean': cv_results['mean_spec'][cls],
        'Std': cv_results['std_spec'][cls]
    })

cv_df_organized = pd.DataFrame(rows)

cv_df_organized = cv_df_organized.sort_values(by=['Class', 'Metric']).reset_index(drop=True)

cv_df_organized.to_csv('cross_validation_results_multiclass_main_tabnet.csv', index=False)

cv_df_organized


,Class,Metric,Mean,Std
0,Addictive disorder,Sensitivity,0.548847,0.090156
1,Addictive disorder,Specificity,0.910374,0.028472
2,Anxiety disorder,Sensitivity,0.597624,0.024378
3,Anxiety disorder,Specificity,0.998746,0.002508
4,Healthy control,Sensitivity,0.770510,0.057006
5,Healthy control,Specificity,0.971818,0.017354
6,Mood disorder,Sensitivity,0.796646,0.069518
7,Mood disorder,Specificity,0.737485,0.029576
8,Obsessive compulsive disorder,Sensitivity,0.826904,0.025848
9,Obsessive compulsive disorder,Specificity,0.998117,0.002514


## | 7. Conclusions, Problems, and Limitations

[need to update]